In [421]:
import pandas as pd
import networkx as nx
import numpy as np

In [422]:
with open('../data/clean_dialog.csv') as f:
    script_df = pd.read_csv(f)

In [423]:
script_df.pony = script_df.pony.apply(lambda x: x.lower())

In [424]:
script_df = script_df[['title','pony']]

In [425]:
interactions = {}
internal = {}

In [426]:
#populate dictionary with pony names
for names in script_df.pony.unique():
    interactions.update({names:{}})

In [427]:
#populate the outer ponies with inner pony dict
for names in script_df.pony.unique():
    internal.update({names:0})

In [428]:
#populate dictionary with pony names
for names in script_df.pony.unique():
    interactions[names].update(internal)

In [429]:
title = script_df.iloc[[0]].title.values[0]
pony = script_df.iloc[[0]].pony.values[0]

for i in range(1,len(script_df)-1):
    #if we are in the same episode
    if title is script_df.iloc[[i]].title.values[0]:#acess title on line i
        
        #check if the current pony and previous pony are differnt
        if pony is not script_df.iloc[[i]].pony.values[0]: #access pony on line i
            interactions[pony][script_df.iloc[[i]].pony.values[0]] = interactions[pony][script_df.iloc[[i]].pony.values[0]] +1
            interactions[script_df.iloc[[i]].pony.values[0]][pony] = interactions[script_df.iloc[[i]].pony.values[0]][pony] +1
    title = script_df.iloc[[i]].title.values[0]
    pony = script_df.iloc[[i]].pony.values[0]


above is the code to calculate all interactions

In [430]:
all_unique_pony = script_df.pony.unique()
null_ponies = []
valid_ponies=[]

In [431]:
for pony in all_unique_pony:
    split_pony = pony.split()
    #append valid names to a list (remove NOT to append INvalid names)
    if ('in' in split_pony or 'and' in split_pony or 'all' in split_pony or 'sans' in split_pony or 'except' in split_pony or 'rest' in split_pony or 'but' in split_pony or 'others' in split_pony or 'ponies' in split_pony):
        null_ponies.append(' '.join(split_pony))
    

In [432]:
for pony in all_unique_pony:
    split_pony = pony.split()
    #append valid names to a list (remove NOT to append INvalid names)
    if not('in' in split_pony or 'and' in split_pony or 'all' in split_pony or 'sans' in split_pony or 'except' in split_pony or 'rest' in split_pony or 'but' in split_pony or 'others' in split_pony or 'ponies' in split_pony):
        valid_ponies.append(' '.join(split_pony))
    

next remove all ponies from the interactions dict

In [433]:
#remove all null ponies from outer dict
for pony in null_ponies:
    interactions.pop(pony)
    

In [434]:
#remove all null ponies from inner dict
for Vpony in valid_ponies:
    for pony in null_ponies:
        interactions[Vpony].pop(pony)
    

remove interactions that have count 0

In [435]:
for pony in interactions:
    for k,v in list(interactions[pony].items()):
        if v == 0:
            del interactions[pony][k]
            

consider the top 101 interactions  
have to remove all the null ponies from the data frame to then count the top valid ponies

In [436]:
pony_df = pd.DataFrame(script_df.pony)

In [437]:
pony_list = list(pony_df.pony)

In [438]:
for x in pony_list:
    if x in null_ponies:
        pony_df = pony_df.drop(pony_df.index[pony_df['pony']==x],inplace=False)

In [439]:
pony_df['number'] = 1

In [440]:
top_101 = list(pony_df.groupby('pony').count().sort_values('number',ascending=False).head(101).index)

now we have to remove all ponies that are not in the top101

In [441]:
for pony in interactions:
    for k in list(interactions[pony]):
        if k not in top_101:
            del interactions[pony][k]

In [442]:
for pony in list(interactions):
    if pony not in top_101:
        del interactions[pony]

remove ponies talking to themselves

In [443]:
try:
    for pony in interactions:
        del interactions[pony][pony]
except:
    pass

In [444]:
interactions

{'twilight sparkle': {'spike': 772,
  'applejack': 384,
  'granny smith': 1,
  'apple bloom': 55,
  'rainbow dash': 458,
  'rarity': 308,
  'fluttershy': 333,
  'pinkie pie': 441,
  'mayor mare': 11,
  'nightmare moon': 14,
  'princess celestia': 129,
  'princess luna': 43,
  'big mcintosh': 3,
  'mrs. cake': 6,
  'mr. cake': 2,
  'snails': 2,
  'snips': 2,
  'trixie': 34,
  'zecora': 30,
  'cheerilee': 7,
  'diamond tiara': 6,
  'scootaloo': 20,
  'sweetie belle': 27,
  'hoity toity': 2,
  'soarin': 1,
  'spitfire': 4,
  'cutie mark crusaders': 7,
  'photo finish': 1,
  'young rainbow dash': 8,
  'discord': 77,
  'pipsqueak': 8,
  'fancy pants': 4,
  'flim': 8,
  'flam': 10,
  'daring do': 1,
  'iron will': 12,
  'garble': 5,
  'shining armor': 61,
  'princess cadance': 73,
  'queen chrysalis': 7,
  'lightning dust': 1,
  'ms. peachbottom': 11,
  'cheese sandwich': 1,
  'maud pie': 10,
  'lord tirek': 13,
  'double diamond': 2,
  'starlight glimmer': 217,
  'sugar belle': 3,
  'prince